# New Data Formatting

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [7]:
fdir = "../COVID-19/csse_covid_19_data/csse_covid_19_daily_reports"

In [13]:
for file in os.listdir(fdir):
    a = file[0]
    if (a.isdigit()):
        date = file[0:-4]
#         print(date)

## Test Multi Index on Single DF

In [14]:
df = pd.read_csv(fdir + "/03-24-2020.csv")

In [17]:
df2 = df.set_index(['Country_Region', 'Province_State', 'Admin2'])

In [25]:
df2.loc['US', "Virginia", "Alexandria"]

FIPS                               51510
Last_Update          2020-03-24 23:37:31
Lat                               38.814
Long_                           -77.0818
Confirmed                              8
Deaths                                 0
Recovered                              0
Active                                 0
Combined_Key    Alexandria, Virginia, US
Name: (US, Virginia, Alexandria), dtype: object

# Combine ALL
so my new thought is that we will stack all of the dataframe, making a column for date, (we can trim the extra columns to save space as well). Then, we will stack them all so we can create a multi-index (lets make it a datetime first though too)

In [30]:
stacked.columns

Index(['Province/State', 'Country/Region', 'Last Update', 'Confirmed',
       'Deaths', 'Recovered', 'date', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Last_Update', 'Lat', 'Long_', 'Active',
       'Combined_Key', 'Latitude', 'Longitude'],
      dtype='object')

In [44]:
# Column renaming dict
col_dict = {
    "Country_Region": "Country/Region",
    "Province_State": "Province/State"
}
cols_to_keep = [
    "date",
    "Province/State",
    "Country/Region",
    "Admin2",
    "Deaths",
    "Recovered",
    "Active",
    "Confirmed"
]
cols_to_drop = [
    "FIPS",
    "Last_Update",
    "Lat",
    "Long_",
    "Combined_Key", 
    "Latitude",
    "Longitude"
]

In [45]:
# For each file
to_stack = []
for file in os.listdir(fdir):
    a = file[0]
    if (a.isdigit()):
        date = file[0:-4]
        df = pd.read_csv(fdir + "/" + file)
        df['date'] = date
        df.rename(columns=col_dict, inplace=True)
#         df = df[cols_to_keep]
        to_stack.append(df)
        
# Open it

# Add a column for the date

# Trim the other columns

# Add to the list for stacking

# Stack them all

# Make multiindex

In [79]:
# Stack them all
stacked = pd.concat(to_stack, axis=0)
stacked = stacked[cols_to_keep]

In [57]:
# stacked date to datetime
stacked['date'] = pd.to_datetime(stacked.date)

In [80]:
sf = stacked.set_index(['date', 'Country/Region', 'Province/State', 'Admin2'])

In [122]:
sf.loc[(slice(None), slice("US"), slice(None), slice(None)), :]

UnsortedIndexError: 'MultiIndex slicing requires the index to be lexsorted: slicing on levels [1], lexsort depth 0'

In [126]:
sf.index.names

FrozenList(['date', 'Country/Region', 'Province/State', 'Admin2'])

In [128]:
sf.sort_index(level=['date', 'Country/Region', 'Province/State'])\
        .loc[(slice(None), slice(None), slice(None), slice("Alexandria")), :]

Deaths  Recovered  \
date       Country/Region Province/State Admin2                              
03-22-2020 US             Alaska         Aleutians East     0.0        0.0   
                                         Aleutians West     0.0        0.0   
                          California     Alameda            0.0        0.0   
                          Colorado       Adams              0.0        0.0   
                                         Alamosa            0.0        0.0   
...                                                         ...        ...   
03-24-2020 US             Virginia       Albemarle          0.0        0.0   
                                         Alexandria         0.0        0.0   
                          Washington     Adams              0.0        0.0   
                          Wisconsin      Adams              0.0        0.0   
                          Wyoming        Albany             0.0        0.0   

                                                         Active  Confirmed  
date       Country/Region Province/State Admin2                             
03-22-2020 US             Alaska         Aleutians East     0.0        0.0  
                                         Aleutians West     0.0        0.0  
                          California     Alameda            0.0       71.0  
                          Colorado       Adams              0.0       18.0  
                                         Alamosa            0.0        0.0  
...                                                         ...        ...  
03-24-2020 US             Virginia       Albemarle          0.0        2.0  
                                         Alexandria         0.0        8.0  
                          Washington     Adams              0.0        1.0  
                          Wisconsin      Adams              0.0        0.0  
                          Wyoming        Albany             0.0        0.0  

[111 rows x 4 columns]

# Getting Serioues
We need functions that take in country and state and return the right stuff

In [150]:
sfd = sf.reset_index()

In [151]:
# Gets global trends
sfd.drop(['Country/Region', 'Province/State', 'Admin2'], axis=1).groupby(['date'], as_index=False).sum()

,date,Deaths,Recovered,Active,Confirmed
0,01-22-2020,17.0,28.0,0.0,555.0
1,01-23-2020,18.0,30.0,0.0,653.0
2,01-24-2020,26.0,36.0,0.0,941.0
3,01-25-2020,42.0,39.0,0.0,1438.0
4,01-26-2020,56.0,52.0,0.0,2118.0
...,...,...,...,...,...
58,03-20-2020,11299.0,87403.0,0.0,272167.0
59,03-21-2020,12973.0,91676.0,0.0,304528.0
60,03-22-2020,14623.0,97243.0,190386.0,337020.0
61,03-23-2020,16497.0,100958.0,215654.0,378287.0


In [161]:
# Get country numbers
latest_date = sfd['date'].max()
sfd[sfd.date == latest_date].drop(['date', 'Province/State', 'Admin2'], axis=1)\
                                            .groupby('Country/Region', as_index=False).sum()

,Country/Region,Deaths,Recovered,Active,Confirmed
0,Afghanistan,1.0,1.0,72.0,74.0
1,Albania,5.0,10.0,108.0,123.0
2,Algeria,19.0,24.0,221.0,264.0
3,Andorra,1.0,1.0,162.0,164.0
4,Angola,0.0,0.0,3.0,3.0
...,...,...,...,...,...
164,Uzbekistan,0.0,0.0,50.0,50.0
165,Venezuela,0.0,15.0,69.0,84.0
166,Vietnam,0.0,17.0,117.0,134.0
167,Zambia,0.0,0.0,3.0,3.0


In [162]:
# Country is selected
country = "US"

In [167]:
# Get trend
sfd.loc[sfd['Country/Region'] == country].drop(['Country/Region', 'Province/State', 'Admin2'], axis=1)\
    .groupby(['date'], as_index=False).sum()

,date,Deaths,Recovered,Active,Confirmed
0,01-22-2020,0.0,0.0,0.0,1.0
1,01-23-2020,0.0,0.0,0.0,1.0
2,01-24-2020,0.0,0.0,0.0,2.0
3,01-25-2020,0.0,0.0,0.0,2.0
4,01-26-2020,0.0,0.0,0.0,5.0
...,...,...,...,...,...
58,03-20-2020,244.0,147.0,0.0,19101.0
59,03-21-2020,307.0,171.0,0.0,25493.0
60,03-22-2020,427.0,0.0,0.0,33746.0
61,03-23-2020,552.0,0.0,0.0,43667.0


In [181]:
# Get next level numbers (state level)
sfd[(sfd['Country/Region'] == country) & (sfd.date == latest_date)] \
        .drop(['date', 'Country/Region', 'Admin2'], axis=1) \
        .groupby('Province/State', as_index=False).sum().sort_values(by='Confirmed', ascending=False).head()

,Province/State,Deaths,Recovered,Active,Confirmed
36,New York,210.0,0.0,0.0,25681.0
34,New Jersey,44.0,0.0,0.0,3675.0
5,California,50.0,0.0,0.0,2538.0
55,Washington,116.0,0.0,0.0,2328.0
26,Michigan,24.0,0.0,0.0,1793.0


In [182]:
# If a state is selected, get the state trend
state = "Virginia"

In [183]:
# Get trend
sfd.loc[sfd['Province/State'] == country].drop(['Country/Region', 'Province/State', 'Admin2'], axis=1)\
    .groupby(['date'], as_index=False).sum()

,date,Deaths,Recovered,Active,Confirmed
0,03-18-2020,0.0,106.0,0.0,1.0
1,03-19-2020,0.0,108.0,0.0,1.0
2,03-20-2020,0.0,147.0,0.0,1.0
3,03-21-2020,0.0,171.0,0.0,1.0


In [184]:
# Get the location list
sfd[(sfd['Province/State'] == state) & (sfd.date == latest_date)] \
        .drop(['date', 'Country/Region', 'Province/State'], axis=1) \
        .groupby('Admin2', as_index=False).sum().sort_values(by='Confirmed', ascending=False).head()

,Admin2,Deaths,Recovered,Active,Confirmed
37,Fairfax,1.0,0.0,0.0,46.0
63,James City,2.0,0.0,0.0,37.0
7,Arlington,0.0,0.0,0.0,36.0
100,Prince William,0.0,0.0,0.0,23.0
70,Loudoun,0.0,0.0,0.0,18.0
